In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
pd.set_option('display.max_colwidth', None)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [11]:
train=train_df
#train=train_df.iloc[1:5800,:] 
#validation=train_df.iloc[5800:,:] 


In [10]:
train

,id,keyword,location,text,target
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
...,...,...,...,...,...
5795,8270,rioting,"Scotland, United Kingdom",@jasalhad @brianboru67 @Jimskiv92 @hijinks1967...,1
5796,8271,rioting,NaN,@RyleeDowns02 @nevaehburton33 if I don't get m...,0
5797,8274,rioting,NaN,RT : Why Sweden Isn't Venezuela: There have be...,1
5798,8276,rioting,Chicora ?? Oakland,@davidolszak or the rioting in happy valley af...,0


In [86]:
from tensorflow import keras

In [87]:
print(keras.__version__)

2.9.0


In [85]:
 print(tf.__version__)

2.9.1


In [12]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, train.target.values, 
                                                  stratify=train.target.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [47]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
test_seq=token.texts_to_sequences(test_df.text.values)

#zero pad the sequences
xtrain_pad = tf.keras.utils.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = tf.keras.utils.pad_sequences(xvalid_seq, maxlen=max_len)
test_pad = tf.keras.utils.pad_sequences(test_seq, maxlen=max_len)
word_index = token.word_index

In [24]:
len(word_index)

22700

In [22]:
%%time
# A simpleRNN without any pretrained embeddings and one dense layer
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                     input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         6810300   
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 6,850,501
Trainable params: 6,850,501
Non-trainable params: 0
_________________________________________________________________
Wall time: 135 ms


In [28]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
96/96 [==============================] - 196s 2s/step - loss: 0.6000 - accuracy: 0.6760
Epoch 2/5
96/96 [==============================] - 179s 2s/step - loss: 0.4499 - accuracy: 0.8197
Epoch 3/5
96/96 [==============================] - 208s 2s/step - loss: 0.3201 - accuracy: 0.9447
Epoch 4/5
96/96 [==============================] - 340s 4s/step - loss: 0.1263 - acc

In [30]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [31]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

48/48 [==============================] - 57s 1s/step
Auc: 0.83%


In [32]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,yvalid)})

In [33]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.8257320622318881}]

In [73]:
#Finally we will predict the test to upload the resuts to kaggle
test_predict=model.predict(test_pad)

102/102 [==============================] - 67s 656ms/step


In [74]:
test_predict.shape

(3263, 1)

In [75]:
test_predict

array([[0.58682007],
       [0.6803706 ],
       [0.926476  ],
       ...,
       [0.7959568 ],
       [0.99326086],
       [0.8571292 ]], dtype=float32)

In [76]:
# We adopt the correct format (1 or 0)
test_predict_format=[1 if value[0]>0.5 else 0 for value in test_predict]
test_predict_format=np.array(test_predict_format)

In [77]:
test_predict_format

array([1, 1, 1, ..., 1, 1, 1])

In [78]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [79]:
sample_submission["target"] = test_predict_format

In [81]:
sample_submission.to_csv("sample_submission_RNN.csv",index=False)